# **Layer 1: Baseline DeiT environment**

DeiT’s baseline training script expects a teacher model name and distillation settings via CLI flags in main.py (e.g., --teacher-model, --teacher-path, --distillation-type).
GitHub
+1

So the “base environment” Layer 1 must include:

DeiT repo (cloned)

PyTorch (Colab default) + GPU

timm installed (for both student and teacher models)

compatibility patches if any (because Colab uses new torch/timm)

Install PyTorch without pinning

In [1]:
!pip -q install --upgrade pip
!pip -q install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.4 MB/s eta 0:00:00


Verify

In [2]:
import torch
print(torch.__version__)
print("CUDA:", torch.cuda.is_available())

2.9.0+cu126
CUDA: True


Clone the baseline repo (official DeiT)

In [3]:
%cd /content
!git clone https://github.com/facebookresearch/deit.git
%cd /content/deit
!grep -n "torch" -n requirements.txt || true

/content
Cloning into 'deit'...
remote: Enumerating objects: 456, done.
remote: Total 456 (delta 0), reused 0 (delta 0), pack-reused 456 (from 1)
Receiving objects: 100% (456/456), 5.73 MiB | 12.15 MiB/s, done.
Resolving deltas: 100% (255/255), done.
/content/deit
1:torch==1.13.1
2:torchvision==0.8.1


Colab Compatibility Fixes

1. torch pin removal

2. timm API changes

3. kwargs popping (pretrained_cfg, cache_dir, etc.)



Patch requirements.txt to remove torch pins

In [4]:
%cd /content/deit

!python - << 'PY'
from pathlib import Path
p = Path("requirements.txt")
lines = p.read_text().splitlines()

filtered = []
removed = []
for line in lines:
    s = line.strip()
    if s.startswith("torch==") or s.startswith("torchvision==") or s.startswith("torchaudio=="):
        removed.append(line)
        continue
    filtered.append(line)

p.write_text("\n".join(filtered) + "\n")
print("✅ Removed these pinned lines:")
for r in removed:
    print("  -", r)

/content/deit
/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
✅ Removed these pinned lines:
  - torch==1.13.1
  - torchvision==0.8.1


Verify Pins are gone!i.e torch==1.13.1 pin was removed

In [5]:
!grep -nE "torch|torchvision|torchaudio" requirements.txt || echo "✅ No torch pins remain"

✅ No torch pins remain


Install the baseline dependencies

In [6]:
pip install "jedi>=0.16,<0.19"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.3 MB/s  0:00:00


In [7]:
!pip -q uninstall -y timm
!pip -q install "jedi>=0.16,<0.19"
!pip -q install timm==0.6.13 submitit
#!pip -q install timm==0.4.12 submitit


Verify

In [8]:
!python -c "import timm; print('timm:', timm.__version__)"

timm: 0.6.13


**Restart the Session**

In [9]:
!python - << 'PY'
from pathlib import Path

p = Path("/usr/local/lib/python3.12/dist-packages/timm/data/__init__.py")
txt = p.read_text()

needle = "OPENAI_CLIP_MEAN"
if needle in txt:
    print("✅ timm.data already mentions OPENAI_CLIP_MEAN; no patch needed.")
else:
    patch = """

# --- Colab patch: expose CLIP normalization constants for older exports ---
try:
    from .constants import OPENAI_CLIP_MEAN, OPENAI_CLIP_STD  # timm versions where defined in constants
except Exception:
    # Standard OpenAI CLIP normalization
    OPENAI_CLIP_MEAN = (0.48145466, 0.4578275, 0.40821073)
    OPENAI_CLIP_STD  = (0.26862954, 0.26130258, 0.27577711)
# --- end patch ---
"""
    p.write_text(txt + patch)
    print("✅ Patched:", p)

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
✅ Patched: /usr/local/lib/python3.12/dist-packages/timm/data/__init__.py


In [10]:
%cd /content/deit
from models import deit_tiny_patch16_224
m = deit_tiny_patch16_224()
print("✅ DeiT model instantiated successfully")

/content/deit
✅ DeiT model instantiated successfully


In [11]:
import torch, timm
print(torch.__version__)
print(timm.__version__)
print(torch.cuda.is_available())

2.9.0+cu126
0.6.13
True


Download Tiny-ImageNet

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
%cd /content
!wget -q http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip

/content


Fix Tiny-ImageNet validation folder

In [14]:
!python - << 'EOF'
import shutil
from pathlib import Path

root = Path("/content/tiny-imagenet-200")
val_dir = root/"val"
img_dir = val_dir/"images"
ann = val_dir/"val_annotations.txt"

with ann.open("r") as f:
    for line in f:
        img, cls = line.strip().split("\t")[:2]
        (val_dir/cls).mkdir(parents=True, exist_ok=True)
        src = img_dir/img
        dst = val_dir/cls/img
        if src.exists():
            shutil.move(str(src), str(dst))

if img_dir.exists():
    shutil.rmtree(img_dir)

print("✅ Tiny-ImageNet val reorganized into class subfolders.")

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `EOF')
✅ Tiny-ImageNet val reorganized into class subfolders.


In [15]:
!find /content/tiny-imagenet-200/val -maxdepth 1 -type d | head

/content/tiny-imagenet-200/val
/content/tiny-imagenet-200/val/n03854065
/content/tiny-imagenet-200/val/n02791270
/content/tiny-imagenet-200/val/n01698640
/content/tiny-imagenet-200/val/n03670208
/content/tiny-imagenet-200/val/n04067472
/content/tiny-imagenet-200/val/n02206856
/content/tiny-imagenet-200/val/n02129165
/content/tiny-imagenet-200/val/n02321529
/content/tiny-imagenet-200/val/n02415577


In [16]:
ls -lah /content/tiny-imagenet-200 | head

total 2.6M
drwxrwxr-x   5 root root 4.0K Feb  9  2015 ./
drwxr-xr-x   1 root root 4.0K Feb  9 02:00 ../
drwxrwxr-x   3 root root 4.0K Dec 12  2014 test/
drwxrwxr-x 202 root root 4.0K Dec 12  2014 train/
drwxrwxr-x 202 root root 4.0K Feb  9 02:01 val/
-rw-rw-r--   1 root root 2.0K Feb  9  2015 wnids.txt
-rw-------   1 root root 2.6M Feb  9  2015 words.txt


Handle timm incompatibilities. Although we can instantiate the model directly, the training script uses timm.create_model(), which injects metadata arguments such as pretrained_cfg and cache_dir.
The original DeiT constructors do not support these arguments, so we remove them
YOUR NOTEBOOK CALL
    |
    v
deit_tiny_patch16_224()          ✅ works (no kwargs)

TRAINING PIPELINE
    |
    v
timm.create_model()
    |
    v
deit_tiny_patch16_224(**kwargs)  ❌ injects extra keys


Patch /content/deit/augment.py (safe compatibility fix)

In [17]:
%cd /content/deit
!python - << 'PY'
from pathlib import Path
p = Path("augment.py")
txt = p.read_text()

old = "from timm.data.transforms import _pil_interp, RandomResizedCropAndInterpolation, ToNumpy, ToTensor"
if old in txt:
    txt = txt.replace(
        old,
        "from timm.data.transforms import RandomResizedCropAndInterpolation, ToNumpy, ToTensor\n"
        "try:\n"
        "    from timm.data.transforms import _pil_interp  # older timm\n"
        "except Exception:\n"
        "    _pil_interp = None  # newer timm doesn't expose this\n"
    )
    p.write_text(txt)
    print("✅ Patched augment.py for timm compatibility.")
else:
    print("ℹ️ Expected import line not found; augment.py may already be patched or different.")

/content/deit
/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
✅ Patched augment.py for timm compatibility.


In [18]:
%cd /content/deit
!rm -f multiteacher_loss.py
!ls -l multiteacher_loss.py || echo "✅ old file removed"

/content/deit
ls: cannot access 'multiteacher_loss.py': No such file or directory
✅ old file removed


In [19]:
%cd /content/deit

from pathlib import Path

code = r'''
from __future__ import annotations

from typing import Dict, List, Optional
import json
from pathlib import Path as _Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import timm


# -----------------------------
# Utilities
# -----------------------------
def normalize_lambdas(lmb: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    """
    Normalize teacher weights so they sum to 1 (supports shape (T,) or (B,T)).
    """
    if lmb.dim() == 1:
        return lmb / lmb.sum().clamp_min(eps)
    return lmb / lmb.sum(dim=-1, keepdim=True).clamp_min(eps)


def fuse_logits(
    teacher_logits: Dict[str, torch.Tensor],
    teacher_order: List[str],
    lambdas: torch.Tensor,
) -> torch.Tensor:
    """
    Weighted sum of teacher logits.
    teacher_logits[k]: (B,C)
    lambdas: (B,T) or (T,)
    returns: (B,C)
    """
    logits_list = [teacher_logits[k] for k in teacher_order]
    stacked = torch.stack(logits_list, dim=1)  # (B,T,C)

    lmb = normalize_lambdas(lambdas).to(stacked.device)
    if lmb.dim() == 1:
        lmb = lmb.unsqueeze(0).expand(stacked.size(0), -1)  # (B,T)

    return (stacked * lmb.unsqueeze(-1)).sum(dim=1)


def kd_soft(student_logits: torch.Tensor, teacher_logits: torch.Tensor, T: float) -> torch.Tensor:
    """
    Standard KL-based soft distillation loss with temperature scaling.
    """
    p_t = F.softmax(teacher_logits / T, dim=-1)
    log_p_s = F.log_softmax(student_logits / T, dim=-1)
    return F.kl_div(log_p_s, p_t, reduction="batchmean") * (T * T)


def kd_hard(student_logits: torch.Tensor, teacher_logits: torch.Tensor) -> torch.Tensor:
    """
    Hard distillation: cross-entropy against teacher argmax.
    """
    return F.cross_entropy(student_logits, teacher_logits.argmax(dim=-1))


# -----------------------------
# Teachers
# -----------------------------
class FrozenTeacherEnsemble(nn.Module):
    """
    Loads a list of timm pretrained teachers and freezes them.
    """
    def __init__(self, teacher_names: List[str], device: torch.device):
        super().__init__()
        self.models = nn.ModuleDict(
            {
                name: timm.create_model(name, pretrained=True, num_classes=1000).eval().to(device)
                for name in teacher_names
            }
        )
        for m in self.models.values():
            for p in m.parameters():
                p.requires_grad_(False)
        self.teacher_order = list(self.models.keys())

    @torch.no_grad()
    def forward(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        return {k: m(x) for k, m in self.models.items()}


# -----------------------------
# Teacher logits mapping: ImageNet-1k -> Tiny-ImageNet (wnid-aligned gather)
# -----------------------------
def build_tiny_imagenet_im1k_indices(
    tiny_root: str,
    class_index_json: str = "/content/imagenet_class_index.json",
) -> torch.Tensor:
    """
    Returns a LongTensor of shape (200,) containing the ImageNet-1k class indices
    corresponding to Tiny-ImageNet wnids.txt ordering.

    Requires torchvision's imagenet_class_index.json (wnid->index via JSON).
    """
    tiny_root_p = _Path(tiny_root)
    wnids_path = tiny_root_p / "wnids.txt"
    if not wnids_path.exists():
        raise FileNotFoundError(f"Could not find Tiny-ImageNet wnids.txt at: {wnids_path}")

    wnids = wnids_path.read_text().strip().splitlines()

    class_index_path = _Path(class_index_json)
    if not class_index_path.exists():
        raise FileNotFoundError(
            f"Missing {class_index_json}. Download it before training.\n"
            "Example:\n"
            "  !wget -q https://raw.githubusercontent.com/pytorch/vision/main/torchvision/models/imagenet_class_index.json "
            f"-O {class_index_json}"
        )

    class_index = json.loads(class_index_path.read_text())
    # class_index: {"0": ["n01440764", "tench"], ...}
    wnid_to_idx = {v[0]: int(k) for k, v in class_index.items()}

    indices: List[int] = []
    missing: List[str] = []
    for w in wnids:
        if w in wnid_to_idx:
            indices.append(wnid_to_idx[w])
        else:
            missing.append(w)

    if missing:
        raise ValueError(
            f"{len(missing)} Tiny-ImageNet wnids were not found in ImageNet-1k mapping. "
            f"First few missing: {missing[:10]}"
        )

    return torch.tensor(indices, dtype=torch.long)


class TeacherLogitMapper(nn.Module):
    """
    Maps ImageNet-1k teacher logits (B,1000) -> Tiny-ImageNet logits (B,200)
    by selecting the 200 corresponding ImageNet indices (gather/index_select).
    """
    def __init__(self, teacher_keys: List[str], im1k_indices: torch.Tensor):
        super().__init__()
        self.teacher_keys = list(teacher_keys)
        self.register_buffer("im1k_indices", im1k_indices)  # (200,)

    def forward(self, teacher_logits: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        out: Dict[str, torch.Tensor] = {}
        idx = self.im1k_indices
        for k, v in teacher_logits.items():
            # v: (B,1000) -> (B,200)
            out[k] = v.index_select(dim=-1, index=idx)
        return out


# -----------------------------
# HDTSE confidence weighting
# -----------------------------
class HDTSEConfidence(nn.Module):
    """
    Computes per-sample teacher weights based on each teacher's confidence
    on the (possibly soft) targets.
    """
    def __init__(self, temp: float = 1.0):
        super().__init__()
        self.temp = float(temp)

    @torch.no_grad()
    def forward(
        self,
        teacher_logits: Dict[str, torch.Tensor],
        teacher_order: List[str],
        targets: torch.Tensor,
    ) -> torch.Tensor:
        stacked = torch.stack([teacher_logits[k] for k in teacher_order], dim=1)  # (B,T,C)
        probs = F.softmax(stacked / self.temp, dim=-1)  # (B,T,C)

        # Hard labels: (B,)
        if targets.dim() == 1:
            idx = targets.to(dtype=torch.long, device=probs.device)
            conf = probs.gather(-1, idx[:, None, None]).squeeze(-1)  # (B,T)
            return normalize_lambdas(conf)

        # Soft labels (mixup/cutmix): (B,C)
        tgt = targets.to(dtype=probs.dtype, device=probs.device)
        conf = (probs * tgt[:, None, :]).sum(dim=-1)  # (B,T)
        return normalize_lambdas(conf)


# -----------------------------
# Multi-teacher distillation loss
# -----------------------------
class MultiTeacherDistillationLoss(nn.Module):
    def __init__(
        self,
        base_criterion,
        student_num_classes: int,
        teacher_names: List[str],
        distillation_type: str = "soft",
        alpha: float = 0.5,
        tau: float = 2.0,
        device=None,
        use_adapter: bool = True,
        hdtse_warmup_epochs: int = 0,
        lambda_log: bool = True,
    ):
        """
        base_criterion: supervised loss (CE or soft-target CE when mixup is enabled)
        distillation_type: "soft" or "hard"
        alpha: final KD weight
        tau: KD temperature
        use_adapter: if True, expects Tiny-ImageNet mapping via set_tiny_root() before training
        hdtse_warmup_epochs: use uniform lambdas until this epoch (exclusive)
        """
        super().__init__()
        self.base_criterion = base_criterion
        self.distillation_type = str(distillation_type)
        self.tau = float(tau)
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # teachers (frozen)
        self.teachers = FrozenTeacherEnsemble(teacher_names, self.device)
        self.teacher_order = list(self.teachers.teacher_order)

        # teacher->student class mapping (ImageNet-1k -> dataset classes)
        self.use_adapter = bool(use_adapter)
        self.adapter: Optional[nn.Module] = None  # created by set_tiny_root()

        # HDTSE teacher weighting
        self.hdtse = HDTSEConfidence()

        # epoch state
        self.epoch: int = 0
        self.hdtse_warmup_epochs = int(hdtse_warmup_epochs)

        # alpha schedule (KD weight ramp)
        self.alpha_final = float(alpha)
        self.alpha_start = 0.0
        self.alpha_ramp_epochs = 20  # default ramp duration

        # lambda logging (epoch-level)
        self.lambda_log = bool(lambda_log)
        self._lambda_sum = torch.zeros(len(self.teacher_order), dtype=torch.float32)
        self._lambda_count = 0

    # ---- Public setters ----
    def set_epoch(self, epoch: int):
        self.epoch = int(epoch)

    def set_alpha_schedule(self, alpha_start: float = 0.0, alpha_ramp_epochs: int = 20):
        self.alpha_start = float(alpha_start)
        self.alpha_ramp_epochs = int(alpha_ramp_epochs)

    def set_tiny_root(self, tiny_root: str, class_index_json: str = "/content/imagenet_class_index.json"):
        """
        Call once (from main.py) after constructing this loss, before training starts.
        Creates the gather-based teacher logits mapper: (B,1000)->(B,C).
        """
        im1k_indices = build_tiny_imagenet_im1k_indices(tiny_root, class_index_json=class_index_json).to(self.device)
        self.adapter = TeacherLogitMapper(self.teacher_order, im1k_indices).to(self.device)

    # ---- Logging ----
    def pop_lambda_stats(self) -> Optional[Dict[str, float]]:
        """
        Returns mean λ per teacher over the epoch, then resets accumulators.
        Call once per epoch from main.py.
        """
        if self._lambda_count <= 0:
            return None

        mean_lmb = (self._lambda_sum / float(self._lambda_count)).tolist()
        out = {f"lambda_{name}": float(v) for name, v in zip(self.teacher_order, mean_lmb)}

        self._lambda_sum.zero_()
        self._lambda_count = 0
        return out

    # ---- Internals ----
    def _uniform_lambdas(self, batch_size: int, device: torch.device) -> torch.Tensor:
        t = len(self.teacher_order)
        return torch.full((batch_size, t), 1.0 / t, device=device, dtype=torch.float32)

    def _alpha_effective(self) -> float:
        if self.alpha_ramp_epochs <= 0:
            return self.alpha_final
        t = min(1.0, float(self.epoch) / float(self.alpha_ramp_epochs))
        return self.alpha_start + t * (self.alpha_final - self.alpha_start)

    # ---- Forward ----
    def forward(self, inputs: torch.Tensor, outputs: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """
        inputs: images (B,3,H,W)
        outputs: student logits (B,C)
        targets: hard labels (B,) or soft labels (B,C) when mixup/cutmix is enabled
        """
        base_loss = self.base_criterion(outputs, targets)

        with torch.no_grad():
            t_logits = self.teachers(inputs)  # dict: teacher -> (B,1000)

        student_C = outputs.shape[-1]
        any_teacher = next(iter(t_logits.values()))
        teacher_C = any_teacher.shape[-1]

        if teacher_C != student_C:
            if self.adapter is None:
                raise RuntimeError(
                f"Teacher logits have {teacher_C} classes but student has {student_C}. "
                "Adapter not initialized. Call criterion.set_tiny_root(args.data_path)."
            )
            t_logits = self.adapter(t_logits)  # dict: teacher -> (B,student_C)

        # ---- Teacher weights (λ) ----
        if self.epoch < self.hdtse_warmup_epochs:
            lambdas = self._uniform_lambdas(outputs.size(0), outputs.device)  # (B,T)
        else:
            lambdas = self.hdtse(t_logits, self.teacher_order, targets)  # (B,T)

        # ---- λ logging ----
        if self.lambda_log:
            batch_mean = lambdas.detach().mean(dim=0).cpu()  # (T,)
            self._lambda_sum += batch_mean * outputs.size(0)
            self._lambda_count += outputs.size(0)

        fused = fuse_logits(t_logits, self.teacher_order, lambdas)  # (B,C)

        kd = kd_soft(outputs, fused, self.tau) if self.distillation_type == "soft" else kd_hard(outputs, fused)

        alpha_eff = self._alpha_effective()
        return (1.0 - alpha_eff) * base_loss + alpha_eff * kd
'''

path = Path("multiteacher_loss.py")
path.write_text(code)

print("File written:", path)
print("File size (bytes):", path.stat().st_size)

/content/deit
File written: multiteacher_loss.py
File size (bytes): 11996


In [20]:
from pathlib import Path
import re, py_compile

MAIN = Path("/content/deit/main.py")
txt = MAIN.read_text()

# ------------------------------------------------------------
# Helpers (line-safe insertions to avoid indentation/newline bugs)
# ------------------------------------------------------------
def fix_broken_import_concatenation():
    global txt
    # Fix exact failure mode:
    txt = txt.replace(
        "from multiteacher_loss import MultiTeacherDistillationLossfrom samplers import RASampler",
        "from multiteacher_loss import MultiTeacherDistillationLoss\nfrom samplers import RASampler"
    )

def ensure_line_after(match_line_regex: str, new_line: str):
    """Insert `new_line` as a full line right AFTER the first line matching regex."""
    global txt
    if new_line.strip() in txt:
        return
    lines = txt.splitlines(True)  # keep line endings
    for i, line in enumerate(lines):
        if re.search(match_line_regex, line):
            # insert after this line
            if not new_line.endswith("\n"):
                new_line2 = new_line + "\n"
            else:
                new_line2 = new_line
            lines.insert(i + 1, new_line2)
            txt = "".join(lines)
            return
    raise RuntimeError(f"Could not find line to insert after: {match_line_regex}")

def ensure_block_after_line(match_line_regex: str, block: str):
    """Insert a multi-line block after first line matching regex."""
    global txt
    # Heuristic: if first unique token already exists, don't re-add
    if "--teacher-models" in block and "--teacher-models" in txt and "--hdtse-warmup-epochs" in txt and "--lambda-log" in txt:
        return
    lines = txt.splitlines(True)
    for i, line in enumerate(lines):
        if re.search(match_line_regex, line):
            if not block.endswith("\n"):
                block2 = block + "\n"
            else:
                block2 = block
            lines.insert(i + 1, block2)
            txt = "".join(lines)
            return
    raise RuntimeError(f"Could not find line to insert block after: {match_line_regex}")

def replace_first(pattern: str, repl: str, flags=re.DOTALL):
    global txt
    m = re.search(pattern, txt, flags)
    if not m:
        return False
    txt = txt[:m.start()] + repl + txt[m.end():]
    return True

def remove_first_line_matching(line_regex: str):
    global txt
    lines = txt.splitlines(True)
    for i, line in enumerate(lines):
        if re.search(line_regex, line):
            del lines[i]
            txt = "".join(lines)
            return True
    return False

# ------------------------------------------------------------
# 0) Repair if prior patch created the exact SyntaxError
# ------------------------------------------------------------
fix_broken_import_concatenation()

# ------------------------------------------------------------
# 1) Ensure MultiTeacherDistillationLoss import (safe line insertion)
# Insert after: from losses import DistillationLoss
# ------------------------------------------------------------
ensure_line_after(
    r"^\s*from\s+losses\s+import\s+DistillationLoss\s*$",
    "from multiteacher_loss import MultiTeacherDistillationLoss"
)

# ------------------------------------------------------------
# 2) Ensure CLI args after --teacher-path
# ------------------------------------------------------------
cli_block = """\
    parser.add_argument('--teacher-models', type=str, default='',
                        help='Comma-separated timm model names for multi-teacher distillation')
    parser.add_argument('--hdtse-warmup-epochs', type=int, default=0,
                        help='Use uniform teacher weights for first N epochs, then enable HDTSE weighting')
    parser.add_argument('--lambda-log', action='store_true', default=False,
                        help='Log mean λ (teacher weights) each epoch for multi-teacher distillation')
"""
ensure_block_after_line(r"^\s*parser\.add_argument\('--teacher-path'", cli_block)

# ------------------------------------------------------------
# 3) Allow finetune + distillation ONLY when multi-teacher is used
# Base guard is:
# if args.distillation_type != 'none' and args.finetune and not args.eval:
#     raise NotImplementedError(...)
# ------------------------------------------------------------
replace_first(
    r"^\s*if\s+args\.distillation_type\s*!=\s*'none'\s+and\s+args\.finetune\s+and\s+not\s+args\.eval\s*:\s*\n\s*raise\s+NotImplementedError\([^\n]*\)\s*$",
    "    if args.distillation_type != 'none' and args.finetune and not args.eval and not getattr(args, 'teacher_models', ''):\n"
    "        raise NotImplementedError(\"Finetuning with distillation not yet supported (single-teacher path)\")\n",
    flags=re.MULTILINE
)

# ------------------------------------------------------------
# 4) Move scheduler creation to AFTER adapter param-group add:
# Remove early: lr_scheduler, _ = create_scheduler(args, optimizer)
# ------------------------------------------------------------
remove_first_line_matching(r"^\s*lr_scheduler,\s*_\s*=\s*create_scheduler\(\s*args\s*,\s*optimizer\s*\)\s*$")

# ------------------------------------------------------------
# 5) Unify distillation region (multi-teacher vs single-teacher)
# We'll replace from "teacher_model = None" up to "output_dir = Path(args.output_dir)"
# This avoids indentation mistakes and prevents teacher_path='' crash.
# ------------------------------------------------------------
m_start = re.search(r"^\s*teacher_model\s*=\s*None\s*$", txt, flags=re.MULTILINE)
m_end   = re.search(r"^\s*output_dir\s*=\s*Path\(args\.output_dir\)\s*$", txt, flags=re.MULTILINE)
if not (m_start and m_end and m_start.start() < m_end.start()):
    raise RuntimeError("Could not locate distillation region anchors (teacher_model=None ... output_dir=Path(...))")

unified = """\
    teacher_model = None

    # -------------------------------
    # Unified single + multi-teacher distillation
    # -------------------------------
    teacher_models_str = getattr(args, 'teacher_models', '').strip()

    if args.distillation_type != 'none' and teacher_models_str:
        teacher_names = [t.strip() for t in teacher_models_str.split(',') if t.strip()]
        print(f"✅ Multi-teacher distillation enabled. Teachers: {teacher_names}")

        criterion = MultiTeacherDistillationLoss(
            base_criterion=criterion,
            student_num_classes=args.nb_classes,
            teacher_names=teacher_names,
            distillation_type=args.distillation_type,
            alpha=args.distillation_alpha,
            tau=args.distillation_tau,
            device=device,
            use_adapter=True,
            hdtse_warmup_epochs=getattr(args, 'hdtse_warmup_epochs', 0),
            lambda_log=getattr(args, 'lambda_log', False),
        )

        # Initialize Tiny-ImageNet wnid -> ImageNet-1k index mapping for teacher logits
        if hasattr(criterion, "set_tiny_root"):
            criterion.set_tiny_root(args.data_path)

        # Optional: alpha ramp if you add args later
        if hasattr(criterion, "set_alpha_schedule") and hasattr(args, "alpha_ramp_epochs"):
            criterion.set_alpha_schedule(
                alpha_start=getattr(args, "alpha_start", 0.0),
                alpha_ramp_epochs=getattr(args, "alpha_ramp_epochs", 20),
            )

    else:
        if args.distillation_type != 'none':
            assert args.teacher_path, 'need to specify teacher-path when using single-teacher distillation'
            print(f"Creating teacher model: {args.teacher_model}")
            teacher_model = create_model(
                args.teacher_model,
                pretrained=False,
                num_classes=args.nb_classes,
                global_pool='avg',
            )
            if args.teacher_path.startswith('https'):
                checkpoint = torch.hub.load_state_dict_from_url(
                    args.teacher_path, map_location='cpu', check_hash=True)
            else:
                checkpoint = torch.load(args.teacher_path, map_location='cpu')
            teacher_model.load_state_dict(checkpoint['model'])
            teacher_model.to(device)
            teacher_model.eval()

        criterion = DistillationLoss(
            criterion, teacher_model, args.distillation_type, args.distillation_alpha, args.distillation_tau
        )

    # Scheduler must be created AFTER all optimizer param groups are finalized
    lr_scheduler, _ = create_scheduler(args, optimizer)
"""

txt = txt[:m_start.start()] + unified + "\n    output_dir = Path(args.output_dir)\n" + txt[m_end.end():]

# ----------------------------
# 2) Ensure loss call uses (samples, outputs, targets)
# ----------------------------
# Patch ONLY the simple 2-arg form if present.
if "criterion(samples, outputs, targets)" not in txt:
    txt = re.sub(
        r"loss\s*=\s*criterion\(\s*outputs\s*,\s*targets\s*\)",
        r"loss = criterion(samples, outputs, targets)",
        txt
    )

# ------------------------------------------------------------
# 6) Insert criterion.set_epoch(epoch) before train_one_epoch
# We add it inside the epoch loop, after sampler.set_epoch if present.
# ------------------------------------------------------------
if "criterion.set_epoch(epoch)" not in txt:
    # If distributed block exists, insert after it
    if re.search(r"^\s*if\s+args\.distributed\s*:\s*\n\s*data_loader_train\.sampler\.set_epoch\(epoch\)\s*$", txt, flags=re.MULTILINE):
        txt = re.sub(
            r"(^\s*if\s+args\.distributed\s*:\s*\n\s*data_loader_train\.sampler\.set_epoch\(epoch\)\s*$)",
            r"\1\n        if hasattr(criterion, 'set_epoch'):\n            criterion.set_epoch(epoch)",
            txt,
            flags=re.MULTILINE,
            count=1
        )
    else:
        # Otherwise put at top of loop
        txt = re.sub(
            r"(^\s*for\s+epoch\s+in\s+range\(args\.start_epoch,\s*args\.epochs\)\s*:\s*$)",
            r"\1\n        if hasattr(criterion, 'set_epoch'):\n            criterion.set_epoch(epoch)",
            txt,
            flags=re.MULTILINE,
            count=1
        )

# ------------------------------------------------------------
# 7) Per-epoch λ logging after train_one_epoch call
# ------------------------------------------------------------
if "print('λ means:'" not in txt:
    txt = re.sub(
        r"(train_stats\s*=\s*train_one_epoch\([\s\S]*?\)\s*)\n",
        r"\1\n\n"
        r"        # Optional: log mean λ per teacher (multi-teacher only)\n"
        r"        if getattr(args, 'lambda_log', False) and hasattr(criterion, 'pop_lambda_stats'):\n"
        r"            lambda_means = criterion.pop_lambda_stats()\n"
        r"            if lambda_means:\n"
        r"                print('λ means:', lambda_means)\n",
        txt,
        count=1
    )

# ------------------------------------------------------------
# Write + compile check
# ------------------------------------------------------------
MAIN.write_text(txt)
py_compile.compile(str(MAIN), doraise=True)
print("✅ Patched main.py written and compiles:", MAIN)


✅ Patched main.py written and compiles: /content/deit/main.py


Before constructing the model, remove those keys from kwargs

In [21]:
from pathlib import Path

p = Path("/content/deit/models.py")
lines = p.read_text().splitlines()

out = []
for line in lines:
    out.append(line)
    if line.strip().startswith("def deit_") and "**kwargs" in line:
        out.append("    # Drop timm-injected kwargs not supported by DeiT")
        out.append("    kwargs.pop('pretrained_cfg', None)")
        out.append("    kwargs.pop('pretrained_cfg_overlay', None)")
        out.append("    kwargs.pop('pretrained_cfg_priority', None)")

p.write_text("\n".join(out) + "\n")
print("✅ models.py patched to drop pretrained_cfg kwargs")


✅ models.py patched to drop pretrained_cfg kwargs


Verify

Fix: Patch /content/deit/models.py to drop pretrained_cfg=...

Patch models.py to also drop cache_dir (and friends)

In [22]:
from pathlib import Path

p = Path("/content/deit/models.py")
lines = p.read_text().splitlines()

# Keys that timm may inject but DeiT constructors don't accept
DROP_KEYS = [
    "cache_dir",
    "hf_hub_id",
    "hf_hub_filename",
    "hf_hub_revision",
]

out = []
for line in lines:
    out.append(line)
    # Right after the comment line we previously inserted, add more pops once per function
    if line.strip() == "# Drop timm-injected kwargs not supported by DeiT":
        for k in DROP_KEYS:
            out.append(f"    kwargs.pop('{k}', None)")

p.write_text("\n".join(out) + "\n")
print("✅ Patched models.py to drop cache_dir/hf_hub* kwargs")


✅ Patched models.py to drop cache_dir/hf_hub* kwargs


In [23]:
!rm -f /content/imagenet_class_index.json
!wget https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json \
  -O /content/imagenet_class_index.json

!python - <<'PY'
import json
p="/content/imagenet_class_index.json"
with open(p,"r",encoding="utf-8") as f:
    obj=json.load(f)
print("Loaded OK. Entries:", len(obj))
print("Example 0:", obj["0"])

--2026-02-09 02:01:08--  https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.15.179.123, 54.231.162.224, 16.15.185.143, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.15.179.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35363 (35K) [application/octet-stream]
Saving to: ‘/content/imagenet_class_index.json’

/content/imagenet_c 100%[===================>]  34.53K   160KB/s    in 0.2s    

2026-02-09 02:01:09 (160 KB/s) - ‘/content/imagenet_class_index.json’ saved [35363/35363]

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
Loaded OK. Entries: 1000
Example 0: ['n01440764', 'tench']


In [24]:
# %cd /content/deit
# !python main.py \
#   --model deit_tiny_patch16_224 \
#   --data-path /content/tiny-imagenet-200 \
#   --pretrained \
#   --epochs 1 \
#   --batch-size 64 \
#   --num_workers 2 \
#   --output_dir /content/deit_runs/smoke_test
# %cd /content/deit
# !python main.py \
#   --model deit_tiny_patch16_224 \
#   --data-path /content/tiny-imagenet-200 \
#   --epochs 1 \
#   --batch-size 128 \
#   --num_workers 4 \
#   --input-size 224 \
#   --opt adamw \
#   --lr 5e-4 \
#   --weight-decay 0.05 \
#   --sched cosine \
#   --aa rand-m9-mstd0.5 \
#   --reprob 0.25 \
#   --remode pixel \
#   --recount 1 \
#   --output_dir /content/deit_runs/tiny_imagenet
### correct one
# %cd /content/deit
# !python main.py \
#  --model deit_tiny_patch16_224 \
#  --data-path /content/tiny-imagenet-200 \
#  --finetune https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth \
#  --epochs 10 \
#  --batch-size 128 \
#  --num_workers 4 \
#  --input-size 224 \
#  --opt adamw \
#  --lr 3e-4 \
#  --weight-decay 0.05 \
#  --sched cosine \
#  --warmup-epochs 1 \
#  --smoothing 0.1 \
#  --aa rand-m7-mstd0.5 \
#  --reprob 0.1 \
#  --drop-path 0.1 \
#  --output_dir /content/deit_runs/tiny_imagenet_5ep
%cd /content/deit
!python main.py \
 --model deit_tiny_patch16_224 \
 --data-path /content/tiny-imagenet-200 \
 --finetune https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth \
 --epochs 100 \
 --batch-size 128 \
 --num_workers 4 \
 --input-size 224 \
 --opt adamw \
 --lr 2.5e-4 \
 --weight-decay 0.05 \
 --sched cosine \
 --warmup-epochs 4 \
 --smoothing 0.1 \
 --aa rand-m6-mstd0.5 \
 --reprob 0.2 \
 --model-ema \
 --model-ema-decay 0.9999 \
 --drop-path 0.05 \
 --mixup 0.2 \
 --cutmix 0.0 \
 --mixup-prob 0.5 \
 --distillation-type soft \
 --distillation-alpha 0.5 \
 --distillation-tau 3.0 \
 --hdtse-warmup-epochs 8 \
 --lambda-log \
 --output_dir /content/deit_runs/tiny_imagenet \
 --teacher-models "swin_base_patch4_window7_224,convnext_base,tf_efficientnetv2_l"
# %cd /content/deit
# !python main.py \
#  --model deit_tiny_patch16_224 \
#  --data-path /content/tiny-imagenet-200 \
#  --finetune https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth \
#  --epochs 10 \
#  --batch-size 128 \
#  --num_workers 4 \
#  --input-size 224 \
#  --opt adamw \
#  --lr 2.5e-4 \
#  --weight-decay 0.05 \
#  --sched cosine \
#  --warmup-epochs 1 \
#  --smoothing 0.1 \
#  --aa rand-m7-mstd0.5 \
#  --reprob 0.1 \
#  --drop-path 0.1 \
#  --distillation-type hard \
# --teacher-model regnety_160 \
# --teacher-path https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth \
#  --output_dir /content/deit_runs/tiny_imagenet_10ep
# %cd /content/deit
# !python main.py \
#  --model deit_tiny_distilled_patch16_224 \
#  --data-path /content/tiny-imagenet-200 \
#  --epochs 10 \
#  --batch-size 128 \
#  --num_workers 4 \
#  --input-size 224 \
#  --opt adamw \
#  --lr 7e-4 \
#  --weight-decay 0.05 \
#  --sched cosine \
#  --warmup-epochs 1 \
#  --smoothing 0.0 \
#  --aa rand-m7-mstd0.5 \
#  --reprob 0.1 \
#  --drop-path 0.0 \
#  --distillation-type hard \
#  --distillation-alpha 0.7 \
#  --teacher-model regnety_160 \
#  --teacher-path https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth \
#  --output_dir /content/deit_runs/deit_tiny_distilled_10ep





Streaming output truncated to the last 5000 lines.
Epoch: [46]  [230/781]  eta: 0:03:05  lr: 0.000040  loss: 1.4579 (1.5188)  time: 0.3339  data: 0.0003  max mem: 6459
Epoch: [46]  [240/781]  eta: 0:03:02  lr: 0.000040  loss: 1.4376 (1.5191)  time: 0.3334  data: 0.0003  max mem: 6459
Epoch: [46]  [250/781]  eta: 0:02:58  lr: 0.000040  loss: 1.4129 (1.5212)  time: 0.3333  data: 0.0003  max mem: 6459
Epoch: [46]  [260/781]  eta: 0:02:55  lr: 0.000040  loss: 1.4691 (1.5250)  time: 0.3332  data: 0.0003  max mem: 6459
Epoch: [46]  [270/781]  eta: 0:02:51  lr: 0.000040  loss: 1.4639 (1.5244)  time: 0.3331  data: 0.0003  max mem: 6459
Epoch: [46]  [280/781]  eta: 0:02:48  lr: 0.000040  loss: 1.4213 (1.5212)  time: 0.3330  data: 0.0003  max mem: 6459
Epoch: [46]  [290/781]  eta: 0:02:45  lr: 0.000040  loss: 1.4175 (1.5228)  time: 0.3332  data: 0.0003  max mem: 6459
Epoch: [46]  [300/781]  eta: 0:02:41  lr: 0.000040  loss: 1.4603 (1.5314)  time: 0.3338  data: 0.0003  max mem: 6459
Epoch: [46]  

# **Layer 2: Base Environment — Teacher Models & Multi-Teacher Adaptations**

Layer 2 extends the baseline DeiT environment to support knowledge distillation from one or more teacher models. This layer is additive: it does not modify the baseline DeiT training loop unless explicitly stated.
It includes
1. Teacher Model Support (Single & Multiple)
2. Teacher Registry / Configuration
3. Multi-Teacher Fusion Mechanism (Adaptation Layer)
4. Distillation Loss Integration